In [26]:
# Enable intellisense
%config IPCompleter.greedy=True

In [27]:
# Import modules
import pandas as pd
import numpy as np
import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt

print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
pd.set_option('display.max_colwidth',1000)
pd.options.display.max_rows = 10000

TensorFlow Version: 2.5.0
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [28]:
import sys
sys.path.append("../IMDB/helpers")
import imdb_preprocess_functions as nist_imdb

In [29]:
dir(nist_imdb)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'file_column',
 'get_fit_data',
 'get_imdb_df_data',
 'get_model_performance',
 'hub',
 'label_column',
 'np',
 'output_result',
 'pd',
 'probability_column',
 'text_column',
 'to_categorical']

In [30]:
# Now load the Stanford IMDB training and test dataset.
[df_train, df_test] = nist_imdb.get_imdb_df_data('../IMDB/data/imdb_master.csv')

The number of rows and columns in the training dataset is: (25000, 5)
Missing values in train dataset:
Unnamed: 0    0
type          0
review        0
label         0
file          0
dtype: int64
Check train class balance
0.0    12500
1.0    12500
Name: label, dtype: int64
The number of rows and columns in the test dataset is: (25000, 5)
Missing values in test dataset:
Unnamed: 0    0
type          0
review        0
label         0
file          0
dtype: int64
Check test class balance
0.0    12500
1.0    12500
Name: label, dtype: int64


In [31]:
# Shuffle the dataset
df = df_train.sample(frac=1, random_state=0)
df.shape

(25000, 5)

In [32]:
SPLIT_TRAIN_SIZE = 20000

In [33]:
df_train_train = df[:SPLIT_TRAIN_SIZE]
df_train_test = df[SPLIT_TRAIN_SIZE:]

In [34]:
df_train_train.shape

(20000, 5)

In [35]:
df_train_test.shape

(5000, 5)

In [36]:
# Check the target class balance
df_train_train[nist_imdb.label_column].value_counts()

0.0    10008
1.0     9992
Name: label, dtype: int64

In [37]:
# Check the target class balance
df_train_test[nist_imdb.label_column].value_counts()

1.0    2508
0.0    2492
Name: label, dtype: int64

In [38]:
file_train_train = 'imdb_train_split_' + str(SPLIT_TRAIN_SIZE) + '.csv'
file_train_test = 'imdb_train_split_'  + str(25000 - SPLIT_TRAIN_SIZE) + '.csv'
df_train_train.to_csv(file_train_train, index=False, columns = [nist_imdb.text_column, nist_imdb.label_column, nist_imdb.file_column])
df_train_test.to_csv(file_train_test, index=False, columns = [nist_imdb.text_column, nist_imdb.label_column, nist_imdb.file_column])